In [3]:
# https://towardsdatascience.com/robust-facial-landmarks-for-occluded-angled-faces-925e465cbf2e

In [4]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [5]:
def get_face_detector(modelFile = "models/res10_300x300_ssd_iter_140000.caffemodel",
                      configFile = "models/deploy.prototxt"):
    """
    Get the face detection caffe model of OpenCV's DNN module
    
    Parameters
    ----------
    modelFile : string, optional
        Path to model file. The default is "models/res10_300x300_ssd_iter_140000.caffemodel".
    configFile : string, optional
        Path to config file. The default is "models/deploy.prototxt".
    Returns
    -------
    model : dnn_Net
    """
    modelFile = "models/res10_300x300_ssd_iter_140000.caffemodel"
    configFile = "models/deploy.prototxt"
    model = cv2.dnn.readNetFromCaffe(configFile, modelFile)
    return model

In [6]:
def find_faces(img, model):
    """
    Find the faces in an image
    
    Parameters
    ----------
    img : np.uint8
        Image to find faces from
    model : dnn_Net
        Face detection model
    Returns
    -------
    faces : list
        List of coordinates of the faces detected in the image
    """
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0,
	(300, 300), (104.0, 177.0, 123.0))
    model.setInput(blob)
    res = model.forward()
    faces = []
    for i in range(res.shape[2]):
        confidence = res[0, 0, i, 2]
        if confidence > 0.5:
            box = res[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            faces.append([x, y, x1, y1])
    return faces

In [7]:
def get_landmark_model(saved_model='models/pose_model'):
    """
    Get the facial landmark model. 
    Original repository: https://github.com/yinguobing/cnn-facial-landmark
    Parameters
    ----------
    saved_model : string, optional
        Path to facial landmarks model. The default is 'models/pose_model'.
    Returns
    -------
    model : Tensorflow model
        Facial landmarks model
    """
    model = keras.models.load_model(saved_model)
    return model

In [8]:
def get_square_box(box):
    """Get a square box out of the given box, by expanding it."""
    left_x = box[0]
    top_y = box[1]
    right_x = box[2]
    bottom_y = box[3]

    box_width = right_x - left_x
    box_height = bottom_y - top_y

    # Check if box is already a square. If not, make it a square.
    diff = box_height - box_width
    delta = int(abs(diff) / 2)

    if diff == 0:                   # Already a square.
        return box
    elif diff > 0:                  # Height > width, a slim box.
        left_x -= delta
        right_x += delta
        if diff % 2 == 1:
            right_x += 1
    else:                           # Width > height, a short box.
        top_y -= delta
        bottom_y += delta
        if diff % 2 == 1:
            bottom_y += 1

    # Make sure box is always square.
    assert ((right_x - left_x) == (bottom_y - top_y)), 'Box is not square.'

    return [left_x, top_y, right_x, bottom_y]

In [33]:
def move_box(box, offset):
        """Move the box to direction specified by vector offset"""
        left_x = box[0] + offset[0]
        top_y = box[1] + offset[1]
        right_x = box[2] + offset[0]
        bottom_y = box[3] + offset[1]
        return [left_x, top_y, right_x, bottom_y]

In [54]:
def detect_marks(img, model, face):
    """
    Find the facial landmarks in an image from the faces
    Parameters
    ----------
    img : np.uint8
        The image in which landmarks are to be found
    model : Tensorflow model
        Loaded facial landmark model
    face : list
        Face coordinates (x, y, x1, y1) in which the landmarks are to be found
    Returns
    -------
    marks : numpy array
        facial landmark points
    """

    offset_y = int(abs((face[3] - face[1]) * 0.1))
    box_moved = move_box(face, [0, offset_y])
    facebox = get_square_box(box_moved)
    
    face_img = img[facebox[1]: facebox[3],
                     facebox[0]: facebox[2]]
    face_img = cv2.resize(face_img, (128, 128))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    
    # # Actual detection.
    predictions = model.signatures["predict"](
        tf.constant([face_img], dtype=tf.uint8))
    
    #######################
    #Scores
    #Ariel
    #scores = model.signatures["probabilities"](
    #    tf.constant([face_img], dtype=tf.dt_float))
    
    # get class prediction
    #out = model.forward()
    #pred_probas = out["detection_out"]
    #print(pred_probas.argmax())
    
    # Otra forma
    #layer_name = 'detection_out'
    #model.summary()
    #idx=0
    #probas=model.get_layer(layer_name).output
    #sh=model.blobs[layer_name].data
    #print(sh)
    #probas = np.array(predictions['probabilities']).flatten()[:136]
    #print(probas.shape)
    ###############################
    
    # Convert predictions to landmarks.
    marks = np.array(predictions['output']).flatten()[:136]
    print(predictions)
    marks = np.reshape(marks, (-1, 2))
    
    marks *= (facebox[2] - facebox[0])
    marks[:, 0] += facebox[0]
    marks[:, 1] += facebox[1]
    marks = marks.astype(np.uint)
    print(marks.shape)
    return marks

def draw_marks(image, marks, color=(0, 255, 0)):
    """
    Draw the facial landmarks on an image
    Parameters
    ----------
    image : np.uint8
        Image on which landmarks are to be drawn.
    marks : list or numpy array
        Facial landmark points
    color : tuple, optional
        Color to which landmarks are to be drawn with. The default is (0, 255, 0).
    Returns
    -------
    None.
    """
    i=0
    mouth_l=[60,61,62,63,64]
    for mark in marks:
        if i in mouth_l:
            cv2.circle(image, (mark[0], mark[1]), 2, color, -1, cv2.LINE_AA)
        i=i+1



In [55]:
face_model = get_face_detector()
landmark_model = get_landmark_model()

cap = cv2.VideoCapture(0)
while(True):
    ret, img = cap.read()
    rects = find_faces(img, face_model)
    
    for rect in rects:
        marks = detect_marks(img, landmark_model, rect)
        draw_marks(img, marks)
    cv2.imshow("image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.11444701, 0.27509302, 0.12326419, 0.3842749 , 0.1400165 ,
        0.4907519 , 0.16489711, 0.5971128 , 0.20890026, 0.6930203 ,
        0.27511817, 0.7771952 , 0.35598078, 0.84773743, 0.4474091 ,
        0.90043974, 0.54445785, 0.9120029 , 0.631618  , 0.88928646,
        0.697575  , 0.8235394 , 0.7538562 , 0.7446835 , 0.79900867,
        0.65816176, 0.8307924 , 0.5645718 , 0.8526006 , 0.46774393,
        0.86506295, 0.36876932, 0.86822695, 0.27094465, 0.21179123,
        0.23805082, 0.26838517, 0.20490152, 0.33987406, 0.19831428,
        0.4135719 , 0.2104754 , 0.4821316 , 0.2363675 , 0.6214482 ,
        0.22905804, 0.6782445 , 0.20180118, 0.7374698 , 0.18483238,
        0.796036  , 0.18760014, 0.8394671 , 0.21448873, 0.554577  ,
        0.31350467, 0.55880606, 0.38761348, 0.563269  , 0.46244207,
        0.569095  , 0.540341  , 0.4782471 , 0.5771473 , 0.5193153 ,
        0.59098506, 0.5617926 , 0.59888595, 0.6000366 ,

        0.7267308 ]], dtype=float32)>}
(68, 2)
{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.11331263, 0.30502236, 0.12281688, 0.41365117, 0.13989598,
        0.51948965, 0.16417658, 0.6254566 , 0.20657626, 0.72123474,
        0.2715251 , 0.80480456, 0.35268414, 0.8747078 , 0.4451452 ,
        0.9267471 , 0.5450923 , 0.93843806, 0.6374415 , 0.91664684,
        0.7105904 , 0.8533612 , 0.77317965, 0.7758989 , 0.8220061 ,
        0.6884428 , 0.8540443 , 0.5921855 , 0.8745427 , 0.49191755,
        0.88674057, 0.3904811 , 0.8909094 , 0.28967834, 0.20261627,
        0.25396562, 0.25756842, 0.21932863, 0.32950866, 0.21157382,
        0.40379477, 0.22277425, 0.47320372, 0.24777189, 0.61770725,
        0.23962167, 0.6786185 , 0.2125907 , 0.7420915 , 0.19666114,
        0.804784  , 0.20184182, 0.85190547, 0.23117891, 0.5483791 ,
        0.3272891 , 0.5514203 , 0.40116125, 0.55480933, 0.47616434,
        0.55954397, 0.5546218 , 0.47012043, 0.5921704 , 0.5112014 ,
        

        0.725664  ]], dtype=float32)>}
(68, 2)
{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.10978104, 0.30029565, 0.1203613 , 0.40952128, 0.13843182,
        0.51570517, 0.16363536, 0.6224373 , 0.20604657, 0.719303  ,
        0.27141705, 0.80427253, 0.35262227, 0.8753754 , 0.44578588,
        0.92819744, 0.5468872 , 0.93963504, 0.639722  , 0.916695  ,
        0.7118989 , 0.8511468 , 0.7722239 , 0.77133983, 0.8190028 ,
        0.6822473 , 0.84968853, 0.58536315, 0.8700029 , 0.48519874,
        0.8817425 , 0.38404018, 0.88523424, 0.28380755, 0.20040116,
        0.24912749, 0.25478435, 0.21372768, 0.32595837, 0.20497218,
        0.40026155, 0.21555874, 0.4693615 , 0.24044582, 0.6154413 ,
        0.23224421, 0.6753394 , 0.2045909 , 0.738153  , 0.18870673,
        0.8003584 , 0.19371486, 0.84719896, 0.22274289, 0.5461395 ,
        0.32042918, 0.54995286, 0.39519444, 0.5536202 , 0.47075635,
        0.55878574, 0.54981303, 0.46935183, 0.58856905, 0.5106055 ,
        

        0.72957605]], dtype=float32)>}
(68, 2)
{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.10582814, 0.30497918, 0.11702593, 0.41393423, 0.13564487,
        0.51990193, 0.1612213 , 0.6260139 , 0.20446016, 0.7221123 ,
        0.2702646 , 0.80630505, 0.35260093, 0.8764961 , 0.44639328,
        0.928651  , 0.54787856, 0.93971694, 0.6413191 , 0.91655135,
        0.7148893 , 0.8513696 , 0.77691483, 0.77196383, 0.824986  ,
        0.6828726 , 0.85597646, 0.5856849 , 0.8752893 , 0.4842987 ,
        0.8862481 , 0.3823137 , 0.88925755, 0.28133467, 0.19669962,
        0.25426677, 0.25057417, 0.21866542, 0.3218033 , 0.20916215,
        0.39614823, 0.2192536 , 0.465849  , 0.24293628, 0.61174643,
        0.23415619, 0.6719122 , 0.20643406, 0.73511493, 0.18988591,
        0.7981533 , 0.1941586 , 0.8459682 , 0.22228754, 0.5430159 ,
        0.322462  , 0.5474335 , 0.39752913, 0.5514554 , 0.47348607,
        0.55728287, 0.5526875 , 0.46844214, 0.59183425, 0.50964785,
        

{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.1059701 , 0.31726402, 0.11775272, 0.42534047, 0.13726965,
        0.53048325, 0.16392598, 0.63499194, 0.20826812, 0.7294727 ,
        0.27413183, 0.81186616, 0.35531542, 0.8807007 , 0.4475893 ,
        0.9315355 , 0.54699796, 0.9415747 , 0.6389431 , 0.9190388 ,
        0.7130343 , 0.8559905 , 0.7763034 , 0.77844703, 0.8262317 ,
        0.69071937, 0.8582582 , 0.59378064, 0.87758577, 0.49139285,
        0.8885671 , 0.38826293, 0.8918675 , 0.2854422 , 0.19220793,
        0.2603722 , 0.24435964, 0.22320338, 0.3150223 , 0.21307763,
        0.38933387, 0.2225284 , 0.45910323, 0.24535698, 0.603182  ,
        0.23648834, 0.6639019 , 0.20850343, 0.7278094 , 0.19134745,
        0.7922715 , 0.19510591, 0.84134585, 0.22331592, 0.5352445 ,
        0.32376528, 0.5398964 , 0.39977276, 0.5436571 , 0.47670853,
        0.5498637 , 0.5564593 , 0.4634004 , 0.595186  , 0.50401616,
        0.6088729 , 0.5474229 , 0.61630446, 0.5876527 ,

...

{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.1031956 , 0.34317875, 0.11608136, 0.44653037, 0.13643211,
        0.54863095, 0.16378307, 0.64797974, 0.20801678, 0.7379917 ,
        0.2743439 , 0.8145863 , 0.35698134, 0.87792647, 0.4491427 ,
        0.92543155, 0.54834896, 0.93360853, 0.641922  , 0.9096526 ,
        0.7194967 , 0.8482331 , 0.7870091 , 0.77343047, 0.83864474,
        0.68788373, 0.8700876 , 0.5920111 , 0.88769656, 0.49093178,
        0.8953159 , 0.3892241 , 0.89454734, 0.2891658 , 0.18135683,
        0.25237706, 0.230754  , 0.21384814, 0.29715735, 0.20116945,
        0.3662516 , 0.20883113, 0.43186748, 0.22922276, 0.59060943,
        0.21623659, 0.64679444, 0.18566364, 0.70915574, 0.16704339,
        0.7726439 , 0.17138574, 0.8223504 , 0.20063482, 0.51934314,
        0.31847316, 0.5245885 , 0.39035055, 0.52958083, 0.46284544,
        0.5362649 , 0.53800833, 0.45609176, 0.5872659 , 0.4956162 ,
        0.59818757, 0.5371241 , 0.60393745, 0.5757656 ,

{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.10425805, 0.34874594, 0.11826883, 0.45141754, 0.13937478,
        0.552978  , 0.16698833, 0.65316445, 0.21032853, 0.7436825 ,
        0.2760236 , 0.8214154 , 0.35932854, 0.88521457, 0.45329186,
        0.9320897 , 0.55579597, 0.9399071 , 0.6515137 , 0.91528094,
        0.72959757, 0.8538477 , 0.79618144, 0.7778231 , 0.8453983 ,
        0.69109523, 0.8756566 , 0.59457564, 0.8923547 , 0.4934106 ,
        0.89931726, 0.3925615 , 0.8983308 , 0.29386675, 0.18160434,
        0.25684693, 0.23051673, 0.21904738, 0.2966149 , 0.20553583,
        0.36579868, 0.21224886, 0.43202707, 0.23177525, 0.5917063 ,
        0.21877319, 0.64914757, 0.18883018, 0.71205866, 0.17090186,
        0.7756451 , 0.17585684, 0.82603836, 0.20382011, 0.51996315,
        0.3187799 , 0.52531767, 0.39142418, 0.5298534 , 0.46431023,
        0.5364312 , 0.5398772 , 0.45597193, 0.58933645, 0.495997  ,
        0.60054624, 0.5384198 , 0.60612804, 0.5780925 ,

{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.10802314, 0.34425887, 0.12177135, 0.4470958 , 0.14276603,
        0.5492327 , 0.17059068, 0.64944994, 0.2145167 , 0.7397258 ,
        0.2802489 , 0.81719315, 0.36241934, 0.8813231 , 0.45524144,
        0.928432  , 0.5566642 , 0.9356935 , 0.6505214 , 0.91105115,
        0.72770876, 0.8502811 , 0.7932657 , 0.77517253, 0.8424681 ,
        0.6893704 , 0.87312245, 0.5939468 , 0.89003634, 0.49329206,
        0.8970973 , 0.39284295, 0.8962397 , 0.29417798, 0.18291752,
        0.25695556, 0.23180516, 0.22005002, 0.2979259 , 0.2071574 ,
        0.36742014, 0.21451628, 0.4340952 , 0.23425032, 0.5908663 ,
        0.22168395, 0.6481794 , 0.19177803, 0.7109196 , 0.17354989,
        0.77451533, 0.17795774, 0.8248794 , 0.20471424, 0.5202472 ,
        0.31971252, 0.52571094, 0.393319  , 0.5299785 , 0.46752667,
        0.5367849 , 0.5440998 , 0.45627338, 0.59117174, 0.49622267,
        0.6026447 , 0.53884673, 0.6084944 , 0.57865465,

        0.73516715]], dtype=float32)>}
(68, 2)
{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.11219577, 0.36309075, 0.13019523, 0.46335375, 0.15511638,
        0.56145537, 0.18682851, 0.6550044 , 0.23313975, 0.7387383 ,
        0.29926848, 0.8093765 , 0.38008088, 0.8662196 , 0.46775687,
        0.9065101 , 0.5627427 , 0.90947235, 0.6495812 , 0.8832199 ,
        0.7184062 , 0.82161725, 0.77585536, 0.7445655 , 0.81866634,
        0.6571666 , 0.8432766 , 0.5631085 , 0.85467535, 0.46452537,
        0.8575257 , 0.36672574, 0.85292983, 0.26982653, 0.18308127,
        0.27537626, 0.2272376 , 0.23058513, 0.29089537, 0.2104223 ,
        0.3589674 , 0.21217136, 0.42432788, 0.22869071, 0.56263095,
        0.20984799, 0.6177281 , 0.17727019, 0.67941844, 0.1595073 ,
        0.7410088 , 0.16471222, 0.78768075, 0.19619247, 0.50540894,
        0.30672514, 0.51419055, 0.37524432, 0.5236685 , 0.44328785,
        0.5335237 , 0.51391745, 0.45638794, 0.56317705, 0.49588764,
        

{'output': <tf.Tensor: shape=(1, 136), dtype=float32, numpy=
array([[0.1224447 , 0.3659984 , 0.1410084 , 0.46448126, 0.16656935,
        0.56090134, 0.19987449, 0.6519835 , 0.24650604, 0.7335188 ,
        0.31180704, 0.8032688 , 0.3909009 , 0.8599361 , 0.4767316 ,
        0.90082884, 0.5694579 , 0.9054146 , 0.6541263 , 0.88001025,
        0.7220066 , 0.82032   , 0.77852976, 0.7444161 , 0.8216442 ,
        0.6588243 , 0.84723806, 0.5659969 , 0.8594885 , 0.46732306,
        0.862623  , 0.36840153, 0.85773444, 0.27063084, 0.1942724 ,
        0.26775092, 0.23808438, 0.22866055, 0.29854584, 0.21216588,
        0.3622609 , 0.21573028, 0.42351145, 0.23211348, 0.568824  ,
        0.21265823, 0.6201112 , 0.17936587, 0.6790408 , 0.16070239,
        0.7388221 , 0.16337943, 0.7865485 , 0.19048026, 0.50866854,
        0.31415156, 0.5173414 , 0.38266736, 0.5266733 , 0.45020992,
        0.53619117, 0.52007866, 0.46337506, 0.5703722 , 0.50150526,
        0.5811789 , 0.5426035 , 0.5851687 , 0.5796963 ,